In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix

1. 데이터 로딩

In [14]:
# Titanic.csv 파일의 raw URL
url = 'https://github.com/Yoon-Kyuhyun/AI-introduction/blob/main/week2/titanic.csv'

# 데이터 로딩
df = pd.read_csv('titanic.csv')

# 데이터 확인
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


2. 결측치 확인

In [4]:
# 결측치 확인
df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

3. 결측치 제거 (Age인 경우 평균나이로)

In [ ]:
# 'Age' 컬럼의 평균값 계산
mean_age = df['Age'].mean()

# 'Age' 컬럼의 결측치를 평균값으로 채우기
df['Age'] = df['Age'].fillna(mean_age)

# 결측치 확인 
print(df.isnull().sum())



PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


4. 레이블 확인 (imbalanced data)

In [8]:
# 레이블 컬럼 분포 확인 (imbalanced 여부)
df['Survived'].value_counts()


0    549
1    342
Name: Survived, dtype: int64

5. 불필요한 컬럼 제거

In [15]:
# 불필요한 컬럼 제거 (예: 'Name', 'Ticket', 'Cabin', 'Embarked')
df = df.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'])

# 데이터 확인
df


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,1,0,3,male,22.0,1,0,7.2500
1,2,1,1,female,38.0,1,0,71.2833
2,3,1,3,female,26.0,0,0,7.9250
3,4,1,1,female,35.0,1,0,53.1000
4,5,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,13.0000
887,888,1,1,female,19.0,0,0,30.0000
888,889,0,3,female,NaN,1,2,23.4500
889,890,1,1,male,26.0,0,0,30.0000


6. 엔코딩 (숫자형으로 변환)

In [16]:
# LabelEncoder 초기화
label_encoder = LabelEncoder()

# 인코딩할 컬럼 리스트 
columns_to_encode = ['Sex']  

# 각 컬럼에 대해 인코딩 수행
for column in columns_to_encode:
    df[column] = label_encoder.fit_transform(df[column])

# 결과 출력
print(df)



     PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare
0              1         0       3    1  22.0      1      0   7.2500
1              2         1       1    0  38.0      1      0  71.2833
2              3         1       3    0  26.0      0      0   7.9250
3              4         1       1    0  35.0      1      0  53.1000
4              5         0       3    1  35.0      0      0   8.0500
..           ...       ...     ...  ...   ...    ...    ...      ...
886          887         0       2    1  27.0      0      0  13.0000
887          888         1       1    0  19.0      0      0  30.0000
888          889         0       3    0   NaN      1      2  23.4500
889          890         1       1    1  26.0      0      0  30.0000
890          891         0       3    1  32.0      0      0   7.7500

[891 rows x 8 columns]


5가지 분류 (RF, DT, LR, KNN, SVM)

In [22]:
# X와 y 분리
X = df.drop(columns=['Survived'])  # 'Survived'는 레이블 컬럼
y = df['Survived']

# 훈련 세트와 테스트 세트로 분할 (80% 훈련, 20% 테스트)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의
models = {
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(),
    'SVM': SVC()
}

# 각 모델의 정확도 및 confusion matrix 출력
for name, model in models.items():
    # 모델 훈련
    model.fit(X_train, y_train)
    
    # 예측
    y_pred = model.predict(X_test)
    
    # 정확도 계산
    accuracy = accuracy_score(y_test, y_pred)
    
    # confusion matrix 계산
    cm = confusion_matrix(y_test, y_pred)
    
    print(f'\n{name} Model')
    print(f'Accuracy: {accuracy:.4f}')
    print('Confusion Matrix:')
    print(cm)
    print('-' * 50)


Random Forest Model
Accuracy: 0.8212
Confusion Matrix:
[[93 12]
 [20 54]]
--------------------------------------------------

Decision Tree Model
Accuracy: 0.7318
Confusion Matrix:
[[84 21]
 [27 47]]
--------------------------------------------------

Logistic Regression Model
Accuracy: 0.7765
Confusion Matrix:
[[89 16]
 [24 50]]
--------------------------------------------------

KNN Model
Accuracy: 0.6592
Confusion Matrix:
[[89 16]
 [45 29]]
--------------------------------------------------

SVM Model
Accuracy: 0.5978
Confusion Matrix:
[[103   2]
 [ 70   4]]
--------------------------------------------------


c:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Admin\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid thi